In [13]:
# career_model_tuning.py (Final Working Version)
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
from lightgbm import LGBMClassifier
from sklearn.metrics import top_k_accuracy_score, classification_report
import optuna
import joblib

# Load and prepare data
data = pd.read_csv("career_dataset.csv")
data['analytical_skill'] = data['reasoning'] * data['numerical_ability']
data['creative_practical'] = data['artistic'] * data['practical']

encoder = LabelEncoder()
y = encoder.fit_transform(data["career"])
X = data.drop("career", axis=1)

# Train-test split & scaling
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, stratify=y, random_state=42
)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

def objective(trial):
    params = {
        'objective': 'multiclass',
        'num_class': 58,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': trial.suggest_int('max_depth', 4, 12),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 100),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 1),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 1),
        'class_weight': 'balanced',
        'n_estimators': 1000,
        'verbosity': -1  # Disable all messages
    }

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = []

    for train_idx, val_idx in cv.split(X_train_scaled, y_train):
        X_tr, X_val = X_train_scaled[train_idx], X_train_scaled[val_idx]
        y_tr, y_val = y_train[train_idx], y_train[val_idx]

        model = LGBMClassifier(**params)
        model.fit(X_tr, y_tr)  # Removed all fit() parameters
        
        val_probs = model.predict_proba(X_val)
        scores.append(top_k_accuracy_score(y_val, val_probs, k=5))

    return np.mean(scores)

# Run optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50, show_progress_bar=True)

# Final training
best_params = study.best_params
best_params.update({
    'n_estimators': 1000,
    'random_state': 42,
    'class_weight': 'balanced',
    'verbosity': -1
})

final_model = LGBMClassifier(**best_params)
final_model.fit(X_train_scaled, y_train)

# Save and evaluate
joblib.dump({'model': final_model, 'scaler': scaler, 'encoder': encoder}, 
            "career_model_pipeline.pkl")
test_probs = final_model.predict_proba(X_test_scaled)
print(f"Final Test Top-5 Accuracy: {top_k_accuracy_score(y_test, test_probs, k=5):.2%}")

[I 2025-04-02 17:20:09,482] A new study created in memory with name: no-name-e8bb647c-c824-4935-a48c-b5093757ab8b


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-04-02 17:27:39,764] Trial 0 finished with value: 0.8039463601532567 and parameters: {'learning_rate': 0.01930245105342609, 'num_leaves': 90, 'max_depth': 12, 'min_child_samples': 76, 'subsample': 0.7167008274477714, 'colsample_bytree': 0.9426413572727468, 'reg_alpha': 0.9937106262919627, 'reg_lambda': 0.16044772452306666}. Best is trial 0 with value: 0.8039463601532567.
[I 2025-04-02 17:29:48,663] Trial 1 finished with value: 0.7690804597701149 and parameters: {'learning_rate': 0.23081358695010204, 'num_leaves': 258, 'max_depth': 11, 'min_child_samples': 23, 'subsample': 0.7184314408342498, 'colsample_bytree': 0.6518897722253529, 'reg_alpha': 0.8918149999306333, 'reg_lambda': 0.47241500569494754}. Best is trial 0 with value: 0.8039463601532567.
[I 2025-04-02 17:32:09,832] Trial 2 finished with value: 0.7939080459770115 and parameters: {'learning_rate': 0.18791363524682603, 'num_leaves': 69, 'max_depth': 6, 'min_child_samples': 58, 'subsample': 0.8915710036736928, 'colsample_byt

[I 2025-04-02 19:05:34,420] Trial 22 finished with value: 0.8054022988505747 and parameters: {'learning_rate': 0.04434362968114848, 'num_leaves': 110, 'max_depth': 7, 'min_child_samples': 89, 'subsample': 0.9339663514566872, 'colsample_bytree': 0.666966312320478, 'reg_alpha': 0.5893496925237491, 'reg_lambda': 0.40719924432313737}. Best is trial 21 with value: 0.806743295019157.
[I 2025-04-02 19:13:26,114] Trial 23 finished with value: 0.805095785440613 and parameters: {'learning_rate': 0.014158618465510983, 'num_leaves': 71, 'max_depth': 8, 'min_child_samples': 66, 'subsample': 0.9687579211847271, 'colsample_bytree': 0.6057027089387745, 'reg_alpha': 0.456528779902366, 'reg_lambda': 0.22015292497688252}. Best is trial 21 with value: 0.806743295019157.
[I 2025-04-02 19:15:59,947] Trial 24 finished with value: 0.8001532567049807 and parameters: {'learning_rate': 0.1048013581141583, 'num_leaves': 43, 'max_depth': 5, 'min_child_samples': 83, 'subsample': 0.7562762824529204, 'colsample_bytre

[I 2025-04-02 20:42:54,203] Trial 44 finished with value: 0.8037547892720307 and parameters: {'learning_rate': 0.031784831454924924, 'num_leaves': 72, 'max_depth': 4, 'min_child_samples': 78, 'subsample': 0.9549726635774958, 'colsample_bytree': 0.7173276753051108, 'reg_alpha': 0.33780953516358325, 'reg_lambda': 0.4372232896191809}. Best is trial 40 with value: 0.8077777777777777.
[I 2025-04-02 20:46:12,153] Trial 45 finished with value: 0.7980459770114943 and parameters: {'learning_rate': 0.08004461271739977, 'num_leaves': 34, 'max_depth': 4, 'min_child_samples': 71, 'subsample': 0.9755554914602506, 'colsample_bytree': 0.6661933206339579, 'reg_alpha': 0.4738814722824539, 'reg_lambda': 0.6789357415078866}. Best is trial 40 with value: 0.8077777777777777.
[I 2025-04-02 20:50:04,323] Trial 46 finished with value: 0.8082375478927203 and parameters: {'learning_rate': 0.024862115144492214, 'num_leaves': 82, 'max_depth': 4, 'min_child_samples': 92, 'subsample': 0.9222437787261959, 'colsample_